**Лабораторная работа 2**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, regexp_replace, year, count, row_number, lower, to_timestamp
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType
import xml.etree.ElementTree as ET

spark = SparkSession.builder \
    .appName("BD_LR2_Ivanova") \
    .getOrCreate()

In [ ]:
xml_file = 'posts_sample.xml'
languages_file = "programming-languages.csv"

In [ ]:
posts_schema = StructType([
    StructField("CreationDate", StringType(), True),
    StructField("Tags", StringType(), True)
])

In [ ]:
# Функция для обработки XML и формирования RDD
def extract_xml_data(file_xml):
    # Парсим XML файл
    document_tree = ET.parse(file_xml)
    root_element = document_tree.getroot()
    records = []

    # Обрабатываем каждую строку с данными
    for element in root_element.findall('row'):
        date_creation = element.attrib.get('CreationDate')  # Получаем дату создания
        tags_field = element.attrib.get('Tags')  # Получаем тег
        if date_creation and tags_field:
            # Убираем символы "<" и ">" из тегов
            clean_tags = regexp_replace(col("Tags"), "[<>]", " ").alias("clean_tags")
            # Разбиваем строки тегов на массив
            tags_array = tags_field.replace('<', ' ').replace('>', ' ').strip().split()
            for tag in tags_array:
                # Добавляем дату и тег в записи
                records.append((date_creation, tag))
    return records

# Создание DataFrame на основе XML данных
xml_data = extract_xml_data(xml_file)
data_frame = spark.createDataFrame(xml_data, schema=StructType([
    StructField("Creation Date", StringType(), True),  # Заголовок для даты создания
    StructField("Tag", StringType(), True)  # Заголовок для тега
]))
data_frame = data_frame.withColumn("Year", year(to_timestamp(col("Creation Date"))))  # Добавление года
data_frame.show(10)  # Отображаем первые 10 строк DataFrame



+--------------------+-------------------+----+
|       Creation Date|                Tag|Year|
+--------------------+-------------------+----+
|2008-07-31T21:42:...|                 c#|2008|
|2008-07-31T21:42:...|     floating-point|2008|
|2008-07-31T21:42:...|    type-conversion|2008|
|2008-07-31T21:42:...|             double|2008|
|2008-07-31T21:42:...|            decimal|2008|
|2008-07-31T22:08:...|               html|2008|
|2008-07-31T22:08:...|                css|2008|
|2008-07-31T22:08:...|internet-explorer-7|2008|
|2008-07-31T23:40:...|                 c#|2008|
|2008-07-31T23:40:...|               .net|2008|
+--------------------+-------------------+----+
only showing top 10 rows



In [ ]:
# Чтение CSV-файла с языками и создание DataFrame
languages_data = spark.read.csv(languages_file, header=True)

# Приведение названий языков к нижнему регистру для унификации
languages_data = languages_data.withColumn("name", lower(col("name")))

# Приведение тегов постов к нижнему регистру для унификации
posts_data = posts_df.withColumn("Tag", lower(col("Tag")))

# Объединение DataFrame постов и языков по совпадающим тегам
# Фильтруем только те записи, где теги соответствуют языкам
matched_posts = posts_data.join(languages_data, posts_data["Tag"] == languages_data["name"], "inner")

# Вывод первых 10 записей результата объединения
matched_posts.show(10)

+--------------------+-----------+----+-----------+--------------------+
|        CreationDate|        Tag|Year|       name|       wikipedia_url|
+--------------------+-----------+----+-----------+--------------------+
|2010-09-23T12:13:...|       java|2010|       java|https://en.wikipe...|
|2010-09-26T17:07:...|        php|2010|        php|https://en.wikipe...|
|2010-09-30T18:27:...|       ruby|2010|       ruby|https://en.wikipe...|
|2010-10-01T11:52:...|          c|2010|          c|https://en.wikipe...|
|2010-10-04T21:05:...|        php|2010|        php|https://en.wikipe...|
|2010-10-06T13:31:...|     python|2010|     python|https://en.wikipe...|
|2010-10-07T20:53:...| javascript|2010| javascript|https://en.wikipe...|
|2010-10-07T23:56:...|applescript|2010|applescript|https://en.wikipe...|
|2010-10-08T14:44:...|        php|2010|        php|https://en.wikipe...|
|2010-10-11T07:54:...|        php|2010|        php|https://en.wikipe...|
+--------------------+-----------+----+-----------+

In [ ]:
tag_counts = filtered_df.groupBy("Year", "Tag").agg(count("*").alias("Count"))

# Определение окна для ранжирования языков по популярности в каждом году
window_spec = Window.partitionBy("Year").orderBy(col("Count").desc())

# Вычисление ранга для каждого языка в каждом году и фильтрация топ-10
top10_df = tag_counts.withColumn("Rank", row_number().over(window_spec)) \
                     .filter(col("Rank") <= 10) \
                     .orderBy("Year", "Rank")
top10_df.show(20)

+----+-----------+-----+----+
|Year|        Tag|Count|Rank|
+----+-----------+-----+----+
|2008|       java|    5|   1|
|2008|       ruby|    4|   2|
|2008|          c|    2|   3|
|2008| javascript|    2|   4|
|2008|        x++|    1|   5|
|2008|     python|    1|   6|
|2008|         io|    1|   7|
|2008|     groovy|    1|   8|
|2008|        php|    1|   9|
|2009|       java|   28|   1|
|2009|     python|   23|   2|
|2009|        php|   22|   3|
|2009| javascript|   12|   4|
|2009|       ruby|    8|   5|
|2009|     delphi|    7|   6|
|2009|          c|    6|   7|
|2009|objective-c|    6|   8|
|2009|    haskell|    4|   9|
|2009|       bash|    3|  10|
|2010|       java|   52|   1|
+----+-----------+-----+----+
only showing top 20 rows



In [ ]:
# Группировка данных по году и тегу для подсчета количества постов
tag_summary = filtered_df.groupBy("Year", "Tag").agg(count("*").alias("Total Count"))

# Определение окна для ранжирования языков по их популярности в пределах каждого года
ranking_window = Window.partitionBy("Year").orderBy(col("Total Count").desc())

# Добавление колонки с рангом для каждого языка в соответствии с его популярностью за год
# Фильтрация для получения только топ-10 языков по популярности
top_languages_per_year = (tag_summary
                          .withColumn("Language Rank", row_number().over(ranking_window))
                          .filter(col("Language Rank") <= 10)
                          .orderBy("Year", "Language Rank"))

# Вывод первых 20 записей с топ-10 языками по популярности за каждый год
top_languages_per_year.show(20)

+----+-----------+-----------+-------------+
|Year|        Tag|Total Count|Language Rank|
+----+-----------+-----------+-------------+
|2008|       java|          5|            1|
|2008|       ruby|          4|            2|
|2008|          c|          2|            3|
|2008| javascript|          2|            4|
|2008|        x++|          1|            5|
|2008|     python|          1|            6|
|2008|         io|          1|            7|
|2008|     groovy|          1|            8|
|2008|        php|          1|            9|
|2009|       java|         28|            1|
|2009|     python|         23|            2|
|2009|        php|         22|            3|
|2009| javascript|         12|            4|
|2009|       ruby|          8|            5|
|2009|     delphi|          7|            6|
|2009|          c|          6|            7|
|2009|objective-c|          6|            8|
|2009|    haskell|          4|            9|
|2009|       bash|          3|           10|
|2010|    

In [ ]:
# Сохранение DataFrame в Parquet
top10_df.write.mode("overwrite").parquet("top_languages.parquet")
from google.colab import files
# Архивация результата
!zip -r top_languages.zip top_languages.parquet
files.download("top_languages.zip")
result_df = spark.read.parquet("top_languages.parquet")
result_df.show(truncate=False, n=500)

updating: top_languages.parquet/ (stored 0%)
updating: top_languages.parquet/._SUCCESS.crc (stored 0%)
updating: top_languages.parquet/_SUCCESS (stored 0%)
  adding: top_languages.parquet/part-00000-1eb50558-b71c-4ce3-bc58-06cdc5223685-c000.snappy.parquet (deflated 36%)
  adding: top_languages.parquet/.part-00000-1eb50558-b71c-4ce3-bc58-06cdc5223685-c000.snappy.parquet.crc (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+----+-----------+-----+----+
|Year|Tag        |Count|Rank|
+----+-----------+-----+----+
|2008|java       |5    |1   |
|2008|ruby       |4    |2   |
|2008|c          |2    |3   |
|2008|javascript |2    |4   |
|2008|x++        |1    |5   |
|2008|python     |1    |6   |
|2008|io         |1    |7   |
|2008|groovy     |1    |8   |
|2008|php        |1    |9   |
|2009|java       |28   |1   |
|2009|python     |23   |2   |
|2009|php        |22   |3   |
|2009|javascript |12   |4   |
|2009|ruby       |8    |5   |
|2009|delphi     |7    |6   |
|2009|c          |6    |7   |
|2009|objective-c|6    |8   |
|2009|haskell    |4    |9   |
|2009|bash       |3    |10  |
|2010|java       |52   |1   |
|2010|php        |46   |2   |
|2010|javascript |44   |3   |
|2010|python     |26   |4   |
|2010|objective-c|23   |5   |
|2010|c          |20   |6   |
|2010|ruby       |12   |7   |
|2010|delphi     |8    |8   |
|2010|applescript|3    |9   |
|2010|r          |3    |10  |
|2011|php        |102  |1   |
|2011|java